In [1]:
import warnings
warnings. simplefilter("ignore")

import pandas as pd
import numpy as np


In [2]:
data=pd.read_json("C:/Users/chenl/Data Science prep/data/song - Song Challenge.json")
data

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00
...,...,...,...,...,...,...
3995,MKVEXMRJGF,122,Louisiana,2015-05-16,Paperback Writer,2015-06-03 08:26:26
3996,HMBEUWPBQI,76,New York,2015-05-08,Reprise / Day in the Life,2015-06-14 15:46:04
3997,KIYDZUFUJN,68,Florida,2015-05-08,Get Back,2015-06-13 13:13:11
3998,BZNJXQSTTL,138,Alabama,2015-05-17,Help!,2015-06-01 17:01:40


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 4000 non-null   object
 1   user_id            4000 non-null   int64 
 2   user_state         4000 non-null   object
 3   user_sign_up_date  4000 non-null   object
 4   song_played        4000 non-null   object
 5   time_played        4000 non-null   object
dtypes: int64(1), object(5)
memory usage: 187.6+ KB


In [7]:
print("unique user # is ",data["user_id"].nunique())
# 196 unique users
# convert time
data["user_sign_up_date"]=pd.to_datetime(data["user_sign_up_date"])
data["time_played"]=pd.to_datetime(data["time_played"])

unique user # is  196


In [14]:
data["song_played"].nunique()

100

# 1 
What are the top 3 and the bottom 3 states in terms of number of users?

In [27]:
# count users groupby states
state_count=data[["user_id","user_state"]].groupby("user_state")["user_id"].nunique().reset_index().sort_values(by="user_id",ascending=False)
print("top 3:",state_count.head(3))
print("bottom 3:",state_count.tail(3))

top 3:     user_state  user_id
25    New York       23
4   California       21
35       Texas       15
bottom 3:    user_state  user_id
22   Nebraska        1
2     Arizona        1
13     Kansas        1


# 2 
What are the top 3 and the bottom 3 states in terms of user engagement? You canchoose how to mathematically define user engagement. 




In [48]:
# total play count , average play count per user
play_count=data[["user_state","time_played"]].groupby("user_state").count().reset_index().sort_values(by="time_played")
engagement=pd.merge(play_count,state_count,how="left",on="user_state")
engagement.columns=["user_state",	"time_played",	"user #"]
engagement["average_play"]=engagement["time_played"]/engagement["user #"]
engagement.sort_values(by="average_play",inplace=True)
print(engagement.head(3))
print(engagement.tail(3))

   user_state  time_played  user #  average_play
0      Kansas            8       1           8.0
3    Virginia           17       2           8.5
14  Minnesota           42       4          10.5
        user_state  time_played  user #  average_play
22  South Carolina           85       3     28.333333
18          Alaska           58       2     29.000000
10        Nebraska           36       1     36.000000


# 3
The CEO wants to send a gift to the first user who signed-up for each state. That is, the first user who signed-up from California, from Oregon, etc. 

In [ ]:
# groupby and find first user base on the sign_up_date
state_1st=data[["user_state","user_id","user_sign_up_date"]].groupby("user_state").

In [55]:
gist_list=data.loc[data.groupby("user_state")["user_sign_up_date"].idxmin(),["user_state","user_id","user_sign_up_date"]]
gist_list.head()

,user_state,user_id,user_sign_up_date
100,Alabama,5,2015-05-01
112,Alaska,106,2015-05-12
368,Arizona,105,2015-05-12
257,Arkansas,78,2015-05-08
397,California,39,2015-05-04


# 4
Build a function that takes as an input any of the songs in the data and returns the most 
likely song to be listened next. That is, if, for instance, a user is currently listening t 
"Eight Days A Week", which song has the highest probability of being played right after  t
by the same user? This is going to be v1 of a song recommendation mod

note: we can do a simple most popular songs to forecast next song, or markov chain to predict next in line, also the collaborative filtering methodsel.

User-based Collaborative Filtering: Recommends songs that these similar users listened to but the current user hasn’t.

    Create a user-song interaction matrix, where rows are users, columns are songs, and values represent interactions;
    Calculate user-to-user similarity using metrics like cosine similarity or Pearson correlation, calculate using the number of users whole listen them together




In [73]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# User-Song matrix
user_song = data.groupby(['user_id', 'song_played'])['id'].count().unstack(fill_value=0)
user_song = (user_song > 0).astype(int)  # Convert to binary interactions
user_song

# Compute user-user similarity
user_similarity = cosine_similarity(user_song)

# Convert to DataFrame for easy access
user_similarity_df = pd.DataFrame(user_similarity, index=user_song.index, columns=user_song.index)

# 3. Define the function to recommend songs for a user
def recommend_songs_for_user(user, user_similarity_df, user_song, k=5):
    """
    Recommend songs for a user based on user-to-user similarity.
    """
    # Find similar users
    similar_users = user_similarity_df.loc[user].sort_values(ascending=False)[1:]  # Exclude the user itself
    
    # Aggregate songs listened to by similar users
    song_scores = user_song.loc[similar_users.index].sum(axis=0)
    
    # Exclude songs already listened to by the current user
    user_songs = user_song.loc[user]
    song_scores = song_scores[user_songs == 0]
    
    # Return the top-k recommendations
    return song_scores.sort_values(ascending=False).head(k)

# 4. Example: Recommend songs for user 1
recommendations = recommend_songs_for_user(user=1, user_similarity_df=user_similarity_df, user_song=user_song, k=5)
print(recommendations)


song_played
Come Together        150
Revolution           141
Get Back             138
Let It Be            109
A Day In The Life    103
dtype: int64


Item-based Collaborative Filtering: Finds songs similar to the currently played song based on how users have interacted with both. Recommends songs that are often played together.

    Create a song-user interaction matrix, where rows are songs, columns are users, and values represent interactions.
    Calculate song-to-song similarity using the same similarity metrics, two songs are considered similar if they are often listened to by the same group of users.

In [65]:
# Song-User matrix
song_user = data.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)
song_user = (song_user > 0).astype(int)
# song_user

#song-song similarity matrix
song_user_norm = normalize(song_user, axis=1)  # normalize the song-user matrix
similarity = np.dot(song_user_norm, song_user_norm.T)  # calculate the similarity matrix
similarity_df = pd.DataFrame(similarity, index=song_user.index, columns=song_user.index)

# similarity_df.head()

# top-k most similar songs
def find_topk(song, similarity, k=1):
    df = similarity.loc[song].sort_values(ascending=False)[1:k + 1].reset_index()
    df = df.rename(columns={'song_played': 'Song', song: 'Similarity'})
    
    return df
df = find_topk(song='Eight Days A Week', similarity=similarity_df, k=10)
df
# the mostly recommend would be Come Together

,Song,Similarity
0,Come Together,0.417399
1,Hey Jude,0.398410
2,While My Guitar Gently Weeps,0.381503
3,Penny Lane,0.380693
4,A Day In The Life,0.377781
5,Let It Be,0.349749
6,Get Back,0.341918
7,Revolution,0.338261
8,I've Got A Feeling,0.335083
9,We Can Work It Out,0.332900


# 5
How would you set up a test to check whether your model works well and is improving 
engagement prolong the listen time


perform a A/B test: randomly split users into two groups, Control group has no recommendation strategy; Experiment group recommend the next song.


one-tailed t-test on 'average #play per hour'
H0: population 'average #play per hour' is same in two groups;
H1: experiment group's population 'average #play per hour' is higher than control group's?